In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [10]:
houses = ['A', 'B', 'C']
xtype_dict = {'r':'raw', 'c':'change', 'l':'last'}

In [11]:
def read_data(house, xtype):
    assert house in houses
    assert xtype in xtype_dict
    
    act_df = pd.read_csv("house{}_act.csv".format(house))
    sensor_df = pd.read_csv("house{}_sensor.csv".format(house))
    X = np.load("X_{}_house{}.npy".format(xtype_dict[xtype], house))
    Y = np.load("Y_house{}.npy".format(house))
    return act_df, sensor_df, X, Y

In [12]:
#read in data
house = 'C'
t = 'l'
act_df,sensor_df,X,Y = read_data(house, t)

In [13]:
#build 4 dict, act to idx, idx to act, sensor to idx, idx to sensor
all_act = list(set(act_df.label)) + [0] #all the act labels
a2i = dict(zip(all_act, range(len(all_act))))
i2a = dict(zip(range(len(all_act)), all_act))

all_sensor = list(set(sensor_df.label)) #all the sensor labels
i2s = dict(zip(range(len(all_sensor)), all_sensor))
s2i = dict(zip(all_sensor, range(len(all_sensor))))

In [14]:
print "# sensors: ", len(all_sensor)
print "# states/acts: ", len(all_act)
print "# timeframes: ", len(X)

# sensors:  21
# states/acts:  18
# timeframes:  26488


In [15]:
miu = np.zeros([len(all_act), len(all_sensor)])
for row in X[Y == all_act[0]]:
    miu[0] += row

## Naive Bayes

In [16]:
#each row represents sensors for a specific action i
#each column is sensor j
def calc_miu(X, Y, all_sensor=all_sensor, all_act=all_act):
    assert len(X)==len(Y)
    
    num_act = len(all_act)
    num_sensor = len(all_sensor)
    miu = np.zeros([num_act, num_sensor])
    for act in all_act:
        sensor = X[Y==act]
        for row in sensor:
            miu[a2i[act]] += row
        total = np.sum(Y==act)
        if total != 0:
            miu[a2i[act]] = miu[a2i[act]]/total
#         else:
#             miu[a2i[act]] = [1e-10]*len(miu[i])
    #normalize
#     for i,row in enumerate(miu):
#         total = np.sum(row)
#         if total != 0:
#             miu[i] = miu[i]/total
#         else:
#             miu[i] = [1e-10]*len(miu[i])
        
    return miu

In [17]:
def max_like(miu, X):
    num_time = len(X)
    num_act = len(miu)
    
    prob = np.zeros((num_time, num_act))
    for i in range(num_time):
        sensor = X[i]
        for j in range(num_act):
            act = miu[j]
            logp = 0
            for k,s in enumerate(sensor):
                if s == 1.:
                    logp += np.log(act[k])
                else:
                    logp += np.log(1-act[k])
            prob[i,j] = logp
    return prob

In [18]:
def mle(matrix):
    est = np.zeros(len(matrix))
    for i,row in enumerate(matrix):
        est[i] = np.argmax(row)
    return est

In [19]:
miu = calc_miu(X,Y)
likes = max_like(miu,X)
result = mle(likes) #note: result is index, not label yet

In [20]:
#convert index to label
res_label = np.array([i2a[e] for e in result])
for act in all_act:
    print "act label: {}, Y count:{}, estimate count:{}".format(act, sum(Y==act), sum(res_label==act))

act label: 1.0, Y count:11961, estimate count:1364
act label: 3.0, Y count:352, estimate count:0
act label: 4.0, Y count:78, estimate count:259
act label: 5.0, Y count:192, estimate count:0
act label: 6.0, Y count:92, estimate count:0
act label: 7.0, Y count:56, estimate count:143
act label: 9.0, Y count:62, estimate count:0
act label: 10.0, Y count:7760, estimate count:7583
act label: 11.0, Y count:88, estimate count:195
act label: 12.0, Y count:9, estimate count:1173
act label: 13.0, Y count:83, estimate count:77
act label: 14.0, Y count:65, estimate count:915
act label: 15.0, Y count:306, estimate count:549
act label: 16.0, Y count:8, estimate count:0
act label: 17.0, Y count:22, estimate count:799
act label: 22.0, Y count:40, estimate count:719
act label: 28.0, Y count:2457, estimate count:12710
act label: 0, Y count:2857, estimate count:2


# Evaluation

$$\text{Precision} = \frac{1}{N}\sum^N_i \frac{TP_i}{TI_i} $$
$$\text{Recall} = \frac{1}{N}\sum^N_i\frac{TP_i}{TT_i} $$
$$\text{F-Measure} = \frac{2 \cdot precision \cdot recal}{precision+recall} $$
$$\text{Accuracy} = \frac{\sum^N_i TP_i}{Total}$$

In [21]:
def precision(pred_label, Y):
    all_label = list(set(Y))
    N = len(all_label)
    res = 0
    for y in list(set(Y)):
        TP = np.sum(pred_label[Y==y]==y)
        TI = np.sum(pred_label==y)
        if TI != 0:
            res += (float(TP)/TI)
    return float(res)/N

def recall(pred_label, Y):
    all_label = list(set(Y))
    N = len(all_label)
    res = 0
    for y in list(set(Y)):
        TP = np.sum(pred_label[Y==y]==y)
        TT = np.sum(Y==y)
        if TT != 0:
            res += float(TP)/TT
    return float(res)/N

def f_score(pred_label, Y):
    p = precision(pred_label, Y)
    r = recall(pred_label, Y)
    return 2*p*r/(p+r)

def accuracy(pred_label, Y):
    res = 0
    all_label = list(set(Y))
    for y in list(set(Y)):
        TP = np.sum(pred_label[Y==y]==y)
        res += TP
    return float(res)/len(Y)

In [22]:
precision(res_label, Y)

0.21998143336511225

In [23]:
recall(res_label, Y)

0.3393190404568084

In [24]:
f_score(res_label, Y)

0.26691874003857746

In [25]:
accuracy(res_label, Y)

0.4359710057384476

In [26]:
# #save miu
np.save("mu{}.npy".format(house), miu)